In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
def swish(x):
    return x*tf.nn.sigmoid(x)

In [2]:
from tensorflow.keras import layers,models
from tensorflow.keras.callbacks import EarlyStopping,CSVLogger,ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.convnext import ConvNeXtSmall

In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar100.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [4]:
x_train=tf.image.resize(x_train,(64,64)).numpy()
x_test=tf.image.resize(x_test,(64,64)).numpy()

In [5]:
x_train=tf.keras.applications.convnext.preprocess_input(x_train)
x_test=tf.keras.applications.convnext.preprocess_input(x_test)

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.25,stratify=y_train)

In [7]:
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [8]:
base_model_small=ConvNeXtSmall(
    include_top=False,
    weights='imagenet',
    input_shape=(64,64,3)
)

198551472/198551472 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [9]:
base_model_small.trainable=False

In [10]:
model=models.Sequential([
    base_model_small,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024,activation='swish'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512,activation='swish'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256,activation='swish'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(100,activation='softmax') 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ convnext_small (Functional)          │ (None, 2, 2, 768)           │      49,454,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 768)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │         787,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 50,931,140 (194.29 MB)

 Trainable params: 1,472,868 (5.62 MB)

 Non-trainable params: 49,458,272 (188.67 MB)

In [11]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=["accuracy"],jit_compile=True)

In [12]:
early_stopper=EarlyStopping(restore_best_weights=True,monitor="val_loss",patience=5)

In [13]:
checkpoint=ModelCheckpoint('best_model.keras',monitor='val_accuracy',save_best_only=True)

In [14]:
csv_logger=CSVLogger('training_log.csv',append=True)

In [15]:
import time
start_time=time.time()

In [16]:
history=model.fit(datagen.flow(x_train,y_train),epochs=20,batch_size=32,callbacks=[early_stopper],validation_data=[x_val,y_val],verbose=1)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1172/1172 ━━━━━━━━━━━━━━━━━━━━ 96s 61ms/step - accuracy: 0.0676 - loss: 4.7851 - val_accuracy: 0.4226 - val_loss: 2.4809
Epoch 2/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 53s 45ms/step - accuracy: 0.2501 - loss: 3.2581 - val_accuracy: 0.4898 - val_loss: 2.0257
Epoch 3/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 53s 45ms/step - accuracy: 0.3174 - loss: 2.7849 - val_accuracy: 0.5231 - val_loss: 1.8375
Epoch 4/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 53s 45ms/step - accuracy: 0.3664 - loss: 2.5256 - val_accuracy: 0.5446 - val_loss: 1.7162
Epoch 5/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 53s 45ms/step - accuracy: 0.3970 - loss: 2.3588 - val_accuracy: 0.5574 - val_loss: 1.6374
Epoch 6/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 53s 45ms/step - accuracy: 0.4157 - loss: 2.2450 - val_accuracy: 0.5707 - val_loss: 1.5833
Epoch 7/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 54s 46ms/step - accuracy: 0.4365 - loss: 2.1552 - val_accuracy: 0.5820 - val_loss: 1.5323
Epoch 8/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 53s 45ms/step - accuracy: 0.4500 - loss: 2.09

In [17]:
head_training_time=time.time()-start_time

In [18]:
base_model_small.trainable=True

In [19]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],jit_compile=True)

In [20]:
start_fine_tune=time.time()

In [21]:
fine_tune_history=model.fit(
    datagen.flow(x_train,y_train,batch_size=32),
    validation_data=(x_val,y_val),
    epochs=10,
    callbacks=[early_stopper,csv_logger],
    verbose=1
)

Epoch 1/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 203s 110ms/step - accuracy: 0.5975 - loss: 1.4347 - val_accuracy: 0.7244 - val_loss: 0.9329
Epoch 2/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 88s 75ms/step - accuracy: 0.6625 - loss: 1.1777 - val_accuracy: 0.7446 - val_loss: 0.8524
Epoch 3/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 89s 75ms/step - accuracy: 0.6876 - loss: 1.0978 - val_accuracy: 0.7614 - val_loss: 0.7970
Epoch 4/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 84s 72ms/step - accuracy: 0.7172 - loss: 0.9863 - val_accuracy: 0.7736 - val_loss: 0.7595
Epoch 5/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 83s 71ms/step - accuracy: 0.7308 - loss: 0.9120 - val_accuracy: 0.7805 - val_loss: 0.7273
Epoch 6/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 83s 71ms/step - accuracy: 0.7490 - loss: 0.8607 - val_accuracy: 0.7869 - val_loss: 0.7077
Epoch 7/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 83s 71ms/step - accuracy: 0.7635 - loss: 0.8044 - val_accuracy: 0.7943 - val_loss: 0.6912
Epoch 8/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 83s 71ms/step - accuracy: 0.7752

In [22]:
fine_tune_time=time.time()-start_fine_tune

In [23]:
test_loss, test_accuracy=model.evaluate(x_test,y_test,verbose=1)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")
print(f"Total training time: {head_training_time + fine_tune_time:.2f} seconds")

313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.8020 - loss: 0.6829

Test Accuracy: 80.41%
Total training time: 2078.22 seconds


In [24]:
val_loss,val_accuracy=model.evaluate(x_val,y_val)
print(f"Validation accuracy: {val_accuracy * 100:.4f}%")

391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.8043 - loss: 0.6490
Validation accuracy: 80.5520%


In [25]:
import numpy as np
y_pred=model.predict(x_test)
y_pred_class=np.argmax(y_pred,axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 30ms/step


In [26]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report


accuracy=accuracy_score(y_test,y_pred_class)
precision=precision_score(y_test,y_pred_class,average='weighted')
recall=recall_score(y_test,y_pred_class,average='weighted')
f1=f1_score(y_test,y_pred_class,average='weighted')


print("\nPerformance Metrics Breakdown for ConvNext Small:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Performance Metrics Breakdown for ConvNext Small:
Accuracy: 0.8041
Precision: 0.8070
Recall: 0.8041
F1-Score: 0.8033


In [27]:
import pandas as pd
results=pd.DataFrame({
    "Model": ["ConvNext Small"],
    "Test Accuracy": [test_accuracy],
    "Validation Accuracy":[val_accuracy],
    "Test Loss": [test_loss],
    "Head Training Time (s)": [head_training_time],
    "Fine-tuning Time (s)": [fine_tune_time],
    "Total Params": [model.count_params()]
})

In [28]:
print(results)

            Model  Test Accuracy  Validation Accuracy  Test Loss  \
0  ConvNext Small         0.8041              0.80552   0.658309   

   Head Training Time (s)  Fine-tuning Time (s)  Total Params  
0             1114.137566            964.081676      50931140  


In [29]:
results.to_csv("ConvNext Small.csv",index=False)
print("\nResults saved to ConvNext Small.csv")


Results saved to ConvNext Small.csv
